# Network compression

Network compression for the computation of EFMs, allowing for greatly reducing the number of model reactions, using some code done by Marco Terzer for EFMTool, and the Stephen Klamt team for the `efmtool_link` python package which is used in CNApy.

In [2]:
import numpy, cobra
import efmtool_link.efmtool_intern as efmtool_intern
import efmtool_link.efmtool_extern as efmtool_extern
import efmtool_link.efmtool4cobra as efmtool4cobra

KeyboardInterrupt: 

In [ ]:
# all parameters here for simpler utilisation
model_file = 'models_storage/HepG2_medium.xml' # model file name
dir_name = './' # parent directory of the directory where mparser_cli is
module_name = './' # directory where mparser_cli is
generated_files_path = '../compressed_HepG2/' # path where the generated files go
model_name = 'new' # prefix name for all generated files (do not put a path there)

: 

In [65]:
# example of filling the values (put absolute paths, not relative paths)
model_file = '/home/maxime/Bureau/Thesis/asp-efm/data/sbml/mitocore/mitocore.xml' # model file name
dir_name = '/home/maxime/Bureau/Thesis/' # parent directory of the directory where mparser_cli is
module_name = 'asp-efm' # directory where mparser_cli is
generated_files_path = '/home/maxime/Bureau/Thesis/asp-efm/data/sbml/mitocore/' # path where the generated files go
model_name = 'mitocore_rd' # prefix name for all generated files (do not put a path there)

In [70]:
model.exchanges

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


[<Reaction EX_m00097x at 0x7f236fc63c10>,
 <Reaction EX_m00157x at 0x7f236fc63b20>,
 <Reaction EX_m00648x at 0x7f236fcc8790>,
 <Reaction EX_m01107x at 0x7f236fcc8e50>,
 <Reaction EX_m01115x at 0x7f236fcc9090>,
 <Reaction EX_m01252x at 0x7f236fcc93f0>,
 <Reaction EX_m01253x at 0x7f236fcc9510>,
 <Reaction EX_m01280x at 0x7f236fcc9870>,
 <Reaction EX_m01285x at 0x7f236fcc9990>,
 <Reaction EX_m01306x at 0x7f236fcca050>,
 <Reaction EX_m01307x at 0x7f236fcca170>,
 <Reaction EX_m01308x at 0x7f236fcca290>,
 <Reaction EX_m01334x at 0x7f236fcca5f0>,
 <Reaction EX_m01365x at 0x7f236fcca950>,
 <Reaction EX_m01369x at 0x7f236fccab90>,
 <Reaction EX_m01370x at 0x7f236fccacb0>,
 <Reaction EX_m01410x at 0x7f236fccb7f0>,
 <Reaction EX_m01419x at 0x7f236fccb910>,
 <Reaction EX_m01433x at 0x7f236fccba30>,
 <Reaction EX_m01445x at 0x7f236fccbd90>,
 <Reaction EX_m01450x at 0x7f236fccbeb0>,
 <Reaction EX_m01513x at 0x7f236fccbfa0>,
 <Reaction EX_m01590x at 0x7f236fcecb50>,
 <Reaction EX_m01596x at 0x7f236fc

In [ ]:
from pathlib import Path
def flux_variability_analysis(model: cobra.Model, loopless=False, fraction_of_optimum=0.0,
                              processes=None, results_cache_dir: Path=None, fva_hash=None, print_func=print):
    # all bounds in the model must be finite because the COBRApy FVA treats unbounded results as errors
    if results_cache_dir is not None:
        fva_hash.update(pickle.dumps((loopless, fraction_of_optimum, model.tolerance))) # integrate solver tolerances?
        if fraction_of_optimum > 0:
            fva_hash.update(pickle.dumps(model.reactions.list_attr("objective_coefficient")))
            fva_hash.update(model.objective_direction.encode())
        file_path = results_cache_dir / (model.id+"_FVA_"+fva_hash.hexdigest())
        fva_result = None
        if Path.exists(file_path):
            try:
                fva_result = pandas.read_pickle(file_path)
                print_func("Loaded FVA result from", str(file_path))
            except:
                print_func("Loading FVA result from", str(file_path), "failed, running FVA.")
        else:
            print_func("No cached result available, running FVA...")
        if fva_result is None:
            fva_result = cobra.flux_analysis.flux_variability_analysis(model, reaction_list=None, loopless=loopless,
                                                             fraction_of_optimum=fraction_of_optimum,
                                                             pfba_factor=None, processes=processes)
            try:
                fva_result.to_pickle(file_path)
                print_func("Saved FVA result to ", str(file_path))
            except:
                print_func("Failed to write FVA result to ", str(file_path))
        return fva_result
    else:
        return cobra.flux_analysis.flux_variability_analysis(model, reaction_list=None, loopless=loopless,
                                                             fraction_of_optimum=fraction_of_optimum,
                                                             pfba_factor=None, processes=processes)

: 

In [ ]:
def fva_model(compressed_model):
    fva_tolerance=1e-9
    with model as fva: # can be skipped when a compressed model is available
        # when include_model_bounds=False modify bounds so that only reversibilites are used?
        # fva.solver = 'glpk_exact' # too slow for large models
        fva.tolerance = fva_tolerance
        fva.objective = model.problem.Objective(0.0)
        if fva.problem.__name__ == 'optlang.glpk_interface':
            # should emulate setting an optimality tolerance (which GLPK simplex does not have)
            fva.solver.configuration._smcp.meth = GLP_DUAL
            fva.solver.configuration._smcp.tol_dj = fva_tolerance
        elif fva.problem.__name__ == 'optlang.coinor_cbc_interface':
            fva.solver.problem.opt_tol = fva_tolerance
        fva_res = flux_variability_analysis(fva, fraction_of_optimum=0.0, processes=1, 
            results_cache_dir=None, fva_hash=None)
    for i in range(fva_res.values.shape[0]): # assumes the FVA results are ordered same as the model reactions
        if abs(fva_res.values[i, 0]) > fva_tolerance: # resolve with glpk_exact?
            compressed_model.reactions[i].lower_bound = fva_res.values[i, 0]
        else:
            compressed_model.reactions[i].lower_bound = 0
        if abs(fva_res.values[i, 1]) > fva_tolerance: # resolve with glpk_exact?
            compressed_model.reactions[i].upper_bound = fva_res.values[i, 1]
        else:
            compressed_model.reactions[i].upper_bound = 0
    return compressed_model

: 

In [ ]:
model = cobra.io.read_sbml_model(model_file)
original_model = model.copy()

: 

In [ ]:
# network compression (currently combination of reaction subsets only)
# IMPORTANT: the model is modified by this function, if you want to keep the full model copy it first
model = fva_model(model)
subT = efmtool4cobra.compress_model_sympy(model) # subT is a matrix for conversion of flux vectors between the full and compressed model
rd = cobra.util.array.create_stoichiometric_matrix(model, array_type='lil')
# model compression makes sure that irreversible reactions always point in the forward direction
rev_rd = [int(r.reversibility) for r in model.reactions]
len(model.reactions)

: 

In [ ]:
print(sum(rev_rd), 'reversible reactions and', len(rev_rd) - sum(rev_rd), 'irreversible')

: 

In [ ]:
import pandas

: 

In [ ]:
# Fonction qui, pour chaque élément itérable l, lui applique la fonction qui en éxtrait l'id de ses éléments.
names = lambda l: list(map(lambda x: x.id, l))

: 

In [ ]:
df = pandas.DataFrame.sparse.from_spmatrix(rd, index=names(model.metabolites), columns=names(model.reactions))
df

: 

In [ ]:
import io
with io.StringIO() as output:
    for column in df.columns:
        dfc = df[column]
        ftime=True
        print(column, ': ', end='', file=output, flush=True)
        for k, v in dfc[dfc < 0].items():
            if ftime:
                ftime=False
            else:
                print('+', end=' ', file=output, flush=True)
            print(-v, k, end=' ', file=output, flush=True)
        print('=' if model.reactions.get_by_id(column).reversibility else '=>', end=' ', file=output, flush=True)
        ftime=True
        for k, v in dfc[dfc > 0].items():
            if ftime:
                ftime=False
            else:
                print('+', end=' ', file=output, flush=True)
            print(v, k, end=' ', file=output, flush=True)
        print(file=output, flush=True)
    lines_r = output.getvalue()
print(lines_r)

: 

In [ ]:
rxn_in_sub = [numpy.where(subT[:, i])[0] for i in range(subT.shape[1])]

: 

In [ ]:
for i, reactions in enumerate(model.reactions):
    print(f'rsub_{i+1}', list(map(lambda x: original_model.reactions[x].id, rxn_in_sub[i])))

: 

In [ ]:
lines = ''
nb_ext = 1
for i, column in enumerate(df.columns):
    dfc = df[column]
    ftime=True
    lines += f'rsub_{i+1}' + ' : '
    for k, v in dfc[dfc < 0].items():
        if ftime:
            ftime=False
        else:
            lines += ' + '
        lines += str(-v) + ' ' + k
    if ftime:
        lines += f'ext_{nb_ext}'
        nb_ext += 1
    lines += ' = ' if model.reactions.get_by_id(column).reversibility else ' => '
    ftime=True
    for k, v in dfc[dfc > 0].items():
        if ftime:
            ftime=False
        else:
            lines += ' + '
        lines += str(v) + ' ' + k
    if ftime:
        lines += f'ext_{nb_ext}'
        nb_ext += 1
    lines += '\n'
lines = '-METEXT\n' + ' '.join(['ext_' + str(i+1) for i in range(nb_ext-1)]) + '\n' + '\n-CAT\n' + lines
print(lines)

: 

In [ ]:
dicto_rsub = {}
biomass = f'rsub_1'
for i, reactions in enumerate(model.reactions[:-1]):
    lm = list(map(lambda x: original_model.reactions[x].id, rxn_in_sub[i]))
    if any(('biomass' in k) or ('BIOMASS' in k) or ('Biomass' in k) for k in lm):
        biomass = f'rsub_{i+1}'
    for r in lm:
        dicto_rsub[r] = f'rsub_{i+1}'
print(biomass)

: 

In [ ]:
biomass = biomass # change biomass here if not good
print(biomass)

: 

In [ ]:
for reaction in model.reactions:
    print(reaction.id, reaction.lower_bound, reaction.upper_bound, reaction.subset_stoich)

: 

# File creation

In [ ]:
from pathlib import Path
from os import chdir
chdir(dir_name)

: 

In [ ]:
import sys
sys.path.append(module_name)
import mparser

: 

In [ ]:
path = generated_files_path

: 

In [ ]:
txtfile = path + f'{model_name}_reduced.txt'

: 

In [ ]:
with open(txtfile, 'w') as f:
    f.writelines(lines)

: 

In [ ]:
ri = lambda i: original_model.reactions[i].id

: 

In [ ]:
subname = lambda i: 'rsub_{}'.format(i+1)
subnamerev = lambda i: 'rsub_{}_rev'.format(i+1)
aq = lambda x: '"{}"'.format(x)
aqrev = lambda x: '"{}_rev"'.format(x)
rev = lambda x: '{}_rev'.format(x)  
lines = 'subset(' + ';'.join([aq(subname(i)) for i in range(len(df.columns))]) + ').\n'
lines += 'subset(' + ';'.join([aq(subnamerev(i)) for i in range(len(df.columns))]) + ').\n'
dicto = {}

for r, ls in enumerate(rxn_in_sub):
    dicto[subname(r)] = {'reacs': [ri(rk) if subT[rk, r] >= 0 else rev(ri(rk)) for rk in ls],
         'coeffs': [abs(subT[rk, r]) for rk in ls],
         'full': [(ri(rk), subT[rk, r]) for rk in ls]}
    if model.reactions.get_by_any(r)[0].reversibility:
        dicto[subnamerev(r)] = {'reacs': [ri(rk) if subT[rk, r] < 0 else rev(ri(rk)) for rk in ls],
             'coeffs': [abs(subT[rk, r]) for rk in ls],
             'full': [(ri(rk), -subT[rk, r]) for rk in ls]}
    for rk in ls:
        fk = subT[rk, r]
        lines += 'coefficient(' + aq(subname(r)) + ',' + ((aq(ri(rk))) if fk >= 0 else aqrev(ri(rk))) + ',' + aq(abs(fk)) + ').\n'
        lines += 'coefficient(' + aq(subnamerev(r)) + ',' + ((aqrev(ri(rk))) if fk >= 0 else aq(ri(rk))) + ',' + aq(abs(fk)) + ').\n'

: 

In [ ]:
lp4file = path + f'{model_name}_reactionSubsets.lp4'
subfile = path + f'{model_name}_reactionSubsets.txt'

: 

In [ ]:
with open(lp4file, 'w') as f:
    f.writelines(lines)

: 

In [ ]:
with open(subfile, 'w') as f:
    f.write(str(dicto))

: 

In [ ]:
mcssubname = lambda i: '"mcs_rsub_{}"'.format(i+1)
mcssubnamerev = lambda i: '"mcs_rsub_{}_rev"'.format(i+1)
aq = lambda x: '"{}"'.format(x)
aqrev = lambda x: '"{}_rev"'.format(x)    
lines = 'subset(' + ';'.join([mcssubname(i) for i in range(len(df.columns))]) + ').\n'
lines += 'subset(' + ';'.join([mcssubnamerev(i) for i in range(len(df.columns))]) + ').\n'

for r, ls in enumerate(rxn_in_sub):
    for rk in ls:
        fk = subT[rk, r]
        lines += 'coefficient(' + mcssubname(r) + ',' + ((aq(ri(rk))) if fk >= 0 else aqrev(ri(rk))) + ',' + aq(abs(fk)) + ').\n'
        lines += 'coefficient(' + mcssubnamerev(r) + ',' + ((aqrev(ri(rk))) if fk >= 0 else aq(ri(rk))) + ',' + aq(abs(fk)) + ').\n'

: 

In [ ]:
lp4file = path + f'{model_name}_mcsReactionSubsets.lp4'     

: 

In [ ]:
with open(lp4file, 'w') as f:
    f.writelines(lines)

: 

In [ ]:
len(original_model.reactions)

: 

In [ ]:
txtfile=f'{path}{model_name}_reduced.txt'
#pklfile=f'{path}{model_name}_reduced.pkl'
aspfile=f'{path}{model_name}_reduced.lp4'
sbmlfile=f'{path}{model_name}_reduced.xml'
#mcs_pklfile=f'{path}{model_name}_reduced_mcs.pkl'
mcs_aspfile=f'{path}{model_name}_reduced_mcs.lp4'
#mcs_sbmlfile=f'{path}{model_name}_reduced_mcs.xml'

: 

### Compressed network files for EFMs computation

In [1]:
#mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.PKL, pklfile, False, [])
mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.ASP, aspfile, False, [])
mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.SBML, sbmlfile, False, [])

NameError: name 'mparser' is not defined

### Compressed network files for MCSs computation with 'biomass' as target

In [ ]:
#mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.PKL, mcs_pklfile, to_dual_mcs=True, target_reactions=[biomass])
mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.ASP, mcs_aspfile, to_dual_mcs=True, target_reactions=[biomass])
#mparser.convert(mparser.conversion.InputFormatType.TXT, txtfile , mparser.conversion.OutputFormatType.SBML, mcs_sbmlfile, to_dual_mcs=True, target_reactions=[biomass])

847 internal metabolites
90 external metabolites
2117 total reactions
80 exchange reactions
482 reversible reactions


/home/jmuller/Documents/Stage_metabo/mparser/meta_network.py:203: UserWarning: No external metabolite
  self.structures_check()
/home/jmuller/Documents/Stage_metabo/mparser/meta_network.py:43: UserWarning: Found reactions with missing overall metabolite production or consumption
  warnings.warn('Found reactions with missing overall metabolite production or consumption')


-- Dual network --
2117 internal metabolites
0 external metabolites
2965 total reactions
80 exchange reactions
1329 reversible reactions
